## Quantify editing for Census Editrons in HEK cells

writes an excel file w/ each sample as a seperate tab

In [1]:
import sys,os, glob
import shutil
import matplotlib as plt
import numpy as np
plt.rcParams['pdf.fonttype'] = 42
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
from Bio import SeqIO
import fuzzysearch
import re
from collections import Counter, OrderedDict
from Bio.Seq import Seq
import xlsxwriter
import gzip
import pathlib
from pathlib import Path

In [2]:
import demultiplexing_module

In [3]:

#EXAMPLE 4
#step through a number of samples, pull out replicates and write them to new fastqs in that file's folder named ???_rep1 or ???_rep2 etc.

##Load key of conditions and files
samples = pd.read_excel("Demultiplexing_Sample_key_AGK_26.xlsx")[["condition",
                                           "run",
                                           "file",
                                           "rep",
                                           "rep_bc"]]

#step though samples
for i in samples.index:
    demultiplexing_module.demultiplex_to_file(samples['run'][i],samples['file'][i],samples['rep'][i],samples['rep_bc'][i]) 

In [2]:
#load retrondb from backup csv for plasmids and umis
#rdb = pd.read_csv(retrondb)
#load key of conditions and files
samples = pd.read_excel("Individual_filekey.xlsx")
samples = samples.set_index("file_name")
samples.head()

,info,plasmid,L_search_seq,R_search_seq,wt_seq,edit_seq,run,rep_bc
file_name,,,,,,,,
msAGK_26_198_rep1,pAGK85_EMX1_R1,85,AGCTGGAGGAGGAAGGGCCTGAGTCCGAGC,CACATCAACCGGTGGCGCATTGCCACGAAG,AGAAGAAGAAGGGCTCCCAT,AGAAGAAAAAGTTCTCCCAT,msAGK_26,TCCTTTCA
msAGK_26_199_rep1,pAGK90_EMX1_R1,90,AGCTGGAGGAGGAAGGGCCTGAGTCCGAGC,CACATCAACCGGTGGCGCATTGCCACGAAG,AGAAGAAGAAGGGCTCCCAT,AGAAGAAAAAGTTCTCCCAT,msAGK_26,TCCTTTCA
msAGK_26_200_rep1,pAGK95_EMX1_all_R1,95,AGCTGGAGGAGGAAGGGCCTGAGTCCGAGC,CACATCAACCGGTGGCGCATTGCCACGAAG,AGAAGAAGAAGGGCTCCCAT,AGAAGAAAAAGTTCTCCCAT,msAGK_26,TCCTTTCA
msAGK_26_200_rep2,pAGK95_EMX1_all_R2,95,AGCTGGAGGAGGAAGGGCCTGAGTCCGAGC,CACATCAACCGGTGGCGCATTGCCACGAAG,AGAAGAAGAAGGGCTCCCAT,AGAAGAAAAAGTTCTCCCAT,msAGK_26,CTGCCCTT
msAGK_26_200_rep3,pAGK95_EMX1_all_R3,95,AGCTGGAGGAGGAAGGGCCTGAGTCCGAGC,CACATCAACCGGTGGCGCATTGCCACGAAG,AGAAGAAGAAGGGCTCCCAT,AGAAGAAAAAGTTCTCCCAT,msAGK_26,CGGTGCAT


In [3]:
samples.index[0].split("_")[2]

'198'

In [4]:
def extract_and_exact_match(sequence, samples, file_name):
        inner_left_flank = fuzzysearch.find_near_matches(samples.loc[file_name,"L_search_seq"],
                                                         sequence,max_l_dist=4)
        inner_right_flank = fuzzysearch.find_near_matches(samples.loc[file_name,"R_search_seq"],
                                                          sequence,max_l_dist=4)
        if len(inner_left_flank) == 1 and len(inner_right_flank) == 1:
            region = sequence[inner_left_flank[0].end:inner_right_flank[0].start]
            wt_or_edit = {"wt": samples.loc[file_name, "wt_seq"],
                          "edit": samples.loc[file_name,"edit_seq"]}
            for possibility in wt_or_edit.keys():
                middle_search_seq = wt_or_edit[possibility]
                if fuzzysearch.find_near_matches(middle_search_seq,region,max_l_dist=0):
                    #change max_l_dist to "1" if 10bp barcode insert
                    return (possibility, region)
            return ('undetermined_no_site_match', region)
        else:
            return ('undetermined_no_flanking_match', None)

def find_file(file_name, run):
    miseq_folder_names = os.listdir('C:\BaseSpace')
    miseq_folder_dict = {}
    for folder in miseq_folder_names:
        miseq_folder_dict[folder.split('-')[0]] = folder
    subfolder_names = os.listdir('C:\BaseSpace\%s' % miseq_folder_dict[run])
    sub_folder_dict = {}
    for folder in subfolder_names:
        try:
            intermediate_folder_key = folder.split('_')[2]
            sub_folder_dict[intermediate_folder_key.split("-")[0]] = folder
        except IndexError: continue
    file_folder = 'C:\BaseSpace\%s\%s' % (miseq_folder_dict[run], sub_folder_dict[file_name.split('_')[2]])
    for file in Path(file_folder).rglob(file_name + "*"):
        file_path = file
        return file
        #return file_path
#     print(file.split('_')[2])
#     print(sub_folder_dict)
#     print(miseq_folder_dict[run])
#     print (file_folder)

def get_editing(file_name, samples, run):
    #input is the fastq and the editing group
    #output is a dictionary of precise editing rate by plasmid name and indel rate
    filepath = find_file(file_name, run)
    print(filepath)
    outcomes_dict = {'undetermined_no_site_match': 0,
                    'undetermined_no_flanking_match': 0,
                    'wt': 0,
                    'edit': 0}
  
    all_reads_str = []
    for seq_record in SeqIO.parse(filepath, "fastq"): 
        all_reads_str.append(str(seq_record.seq))
    read_counter = Counter(all_reads_str)
#     print (Region_dict_specific)
    for read in read_counter:  
    
#     #S_number = file_name.split("_")[2]
#     #fastq_reads = '%s_S%s_L001_R1_001.fastq.gz' %(file_name, S_number)
# #     fastq_reads = 'S%s.fastq' %(file_name, S_number)
# #     print(fastq_reads)
#     all_reads_str = []
#     #adding gzip function... can get rid of this now
# #     with gzip.open(fastq_reads,"rt") as handle:
#     for seq_record in SeqIO.parse(handle, "fastq"): 
#         all_reads_str.append(str(seq_record.seq))
#     read_counter = Counter(all_reads_str)

# #     print (Region_dict_specific)
#     for read in read_counter:
        # get result of extract_and_exact_match
        # (either wt, edit, undetermined_no_site_match, or undetermined_no_flanking_match)
        site_match_result = extract_and_exact_match(read, samples, file_name)[0]
        # adding the number of reads to our counter for that result
        outcomes_dict[site_match_result] = outcomes_dict[site_match_result] + read_counter[read]
    return (outcomes_dict)

In [5]:
writer = pd.ExcelWriter('Individual_Processed.xlsx', engine='xlsxwriter')
for file_name in samples.index.values:
    print(file_name)
    run = ""
    outcomes = get_editing(file_name, samples, samples.loc[file_name, "run"])
  
    no_region = outcomes['undetermined_no_flanking_match']
    del outcomes['undetermined_no_flanking_match']
    region_seqs = sum(outcomes.values())
    print ('%s sequences do not match region (out of %s total sequences)' %(no_region,region_seqs+no_region) +'\n')
    outcomes_plus = {}  #add percent editing
    for key in outcomes:
        try:
            outcomes_plus[key] = [outcomes[key],(float(outcomes[key])/region_seqs)*100]
        except ZeroDivisionError:
            print ('no data to analyze' + '\n')
            break
    sample_data = pd.DataFrame.from_dict(outcomes_plus, orient='index',columns=['raw read count','percent of reads'])
    sorted_data = sample_data.sort_index(ascending=True)
    # sorted_data.index.name = samples.loc[i]['info']
    sorted_data.to_excel(writer, sheet_name=file_name)
    for column in sorted_data:
        column_length = max(sorted_data[column].astype(str).map(len).max(), len(column))
        col_idx = sorted_data.columns.get_loc(column)
        writer.sheets[file_name].set_column(col_idx, col_idx, column_length)
    print (sorted_data)
    print ('\n')
writer.close()

msAGK_26_198_rep1
C:\BaseSpace\msAGK_26\msAGK_26_198-ds.b0405afdef3e406c8120ababb36abc7f\msAGK_26_198_rep1.fastq
279516 sequences do not match region (out of 487800 total sequences)

                            raw read count  percent of reads
edit                                  7673          3.683912
undetermined_no_site_match           77673         37.291871
wt                                  122938         59.024217


msAGK_26_199_rep1
C:\BaseSpace\msAGK_26\msAGK_26_199-ds.d9b5745cf508437fabcfd9bcdf50db4d\msAGK_26_199_rep1.fastq
291482 sequences do not match region (out of 454712 total sequences)

                            raw read count  percent of reads
edit                                  5911          3.621271
undetermined_no_site_match           83462         51.131532
wt                                   73857         45.247197


msAGK_26_200_rep1
C:\BaseSpace\msAGK_26\msAGK_26_200-ds.50e7e776c4f14b9481ee3b0e404ff44d\msAGK_26_200_rep1.fastq
56034 sequences do not match 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
sns.set()
sns.set_style("ticks")
from matplotlib.colors import LogNorm

import matplotlib
matplotlib.rcParams['svg.fonttype'] = 'none'

In [7]:
# integrate data --> it's in sheets idk why
data = pd.DataFrame(columns=["run_id", "edit_raw_read_count",
                             "undetermined_no_site_match_raw_read_count",
                             "wt_raw_read_count", "edit_percent",
                             "undetermined_no_site_match_percent",
                             "wt_percent"])

lib_filepath = "Individual_Processed.xlsx"
lib_dict = pd.read_excel(lib_filepath, sheet_name=None)
for run_id in lib_dict.keys():
    run_data = lib_dict[run_id]
    run_data_dict = {"run_id": run_id,
                     "edit_raw_read_count": run_data.loc[run_data["Unnamed: 0"] == "edit", "raw read count"][0],
                     "undetermined_no_site_match_raw_read_count": run_data.loc[run_data["Unnamed: 0"] == "undetermined_no_site_match", "raw read count"].values[0],
                     "wt_raw_read_count": run_data.loc[run_data["Unnamed: 0"] == "wt", "raw read count"].values[0],
                     "edit_percent": run_data.loc[run_data["Unnamed: 0"] == "edit", "percent of reads"].values[0],
                     "wt_percent": run_data.loc[run_data["Unnamed: 0"] == "wt", "percent of reads"].values[0],
                     "undetermined_no_site_match_percent": run_data.loc[run_data["Unnamed: 0"] == "undetermined_no_site_match",
                                                                        "percent of reads"].values[0]}
    data = pd.concat([data, pd.DataFrame([run_data_dict])], ignore_index=True)

C:\Users\asim.khan\AppData\Local\Temp\ipykernel_9312\2459494118.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([run_data_dict])], ignore_index=True)


In [8]:
data.to_csv("Individual_Processed_Clean.csv")